In [1]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os
import pandas as pd
import cvxpy as cp
import numpy as np

from level1.functions import f_returns_on_df, f_mu_on_df, f_sigma_on_df
from portfolio_utils import load_datas, f_yield, f_volatility
from level3.functions import boostrap_sample_df
from level2.functions import optimize

# Charger les données
df = load_datas()

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2609 entries, 2015-01-01 to 2024-12-31
Columns: 190 entries, AAPL to XEL
dtypes: float64(190)
memory usage: 3.8 MB


In [3]:
set(df.index.year)

{2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024}

In [4]:
c = 0.001
K = 2
re = f_returns_on_df(df)
mu = f_mu_on_df(re)
Sigma = f_sigma_on_df(re)
num_assets = len(mu)
w0 = np.array(len(mu) * [0])
w0[2] = 1

In [5]:
frontier_yields, frontier_volatility, frontier_cost, frontier_weights = optimize(mu, Sigma, w0, K, population_size=300, generations=100, c=c)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      300 |     15 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |      600 |     25 |  0.000000E+00 |  0.000000E+00 |  0.4979130913 |         ideal
     3 |      900 |     41 |  0.000000E+00 |  0.000000E+00 |  0.1319883950 |         nadir
     4 |     1200 |     55 |  0.000000E+00 |  0.000000E+00 |  0.0303245929 |         ideal
     5 |     1500 |     97 |  0.000000E+00 |  0.000000E+00 |  0.0215190476 |             f
     6 |     1800 |    154 |  0.000000E+00 |  0.000000E+00 |  0.0027005839 |         nadir
     7 |     2100 |    243 |  0.000000E+00 |  0.000000E+00 |  0.0058469868 |             f
     8 |     2400 |    300 |  0.000000E+00 |  0.000000E+00 |  0.0053543680 |             f
     9 |     2700 |    300 |  0.000000E+00 |  0.000000E+00 |  0.0053044724 |             f
    10 |     3000 |    300 |  0.000000E+00 |  0.000000E+00 |  0.0044391171 |             f

In [6]:
year_params = {}
for year, b in boostrap_sample_df(df).items():
    re_y = f_returns_on_df(b)
    mu_y = f_mu_on_df(re_y)
    Sigma_y = f_sigma_on_df(re_y)
    year_params[year] = (mu_y, Sigma_y)


                AAPL      MSFT      NVDA      AVGO       AMD      INTC  \
Date                                                                     
2015-01-05 -0.028576 -0.009238 -0.017035 -0.016115 -0.003752 -0.011340   
2015-01-06  0.000094 -0.014786 -0.030787 -0.023006 -0.011342 -0.018813   
2015-01-07  0.013924  0.012625 -0.002609  0.026655 -0.019195  0.020759   
2015-01-08  0.037703  0.028994  0.036927  0.048766  0.011561  0.018430   
2015-01-09  0.001072 -0.008441  0.004020  0.010638  0.007634  0.001906   
...              ...       ...       ...       ...       ...       ...   
2024-12-23  0.003060 -0.003097  0.036232  0.053709  0.044222  0.034243   
2024-12-24  0.011413  0.009330  0.003930  0.031060  0.013472  0.009852   
2024-12-27 -0.013331 -0.017453 -0.021089 -0.014822  0.001039 -0.006873   
2024-12-30 -0.013352 -0.013328  0.003497 -0.025854 -0.022211 -0.023929   
2024-12-31 -0.007083 -0.007869 -0.023549 -0.016003 -0.013568  0.011538   

                QCOM       TXN       

In [7]:
def evaluate_portfolio_over_years(w, year_params):
    rets = []
    vols = []
    for mu_y, Sigma_y in year_params.values():
        r = f_yield(w, mu_y)
        v = f_volatility(w, Sigma_y)
        rets.append(r)
        vols.append(v)
    return np.array(rets), np.array(vols)

std_yields = []
std_vols = []

for w in frontier_weights:
    rets, vols = evaluate_portfolio_over_years(w, year_params)
    std_yields.append(np.std(rets))   # f4 = instabilité rendement
    std_vols.append(np.std(vols))


In [10]:
def normalize(x):
    x = np.array(x)
    return (x - x.min()) / (x.max() - x.min() + 1e-12) # 1e-12 to avoid zero division

# On normalise pour avoir la meme echelle

score = (
    0.4 * normalize(frontier_volatility) +   # niveau de risque
    0.1 * normalize(frontier_cost) +          # coûts
    0.4 * normalize(std_yields) +             # instabilité rendement
    0.1 * normalize(std_vols)                 # instabilité risque
)

In [11]:
best_idx = np.argmin(score)
w_robust = frontier_weights[best_idx]
print(f"Portefeuille robuste sélectionné :")
print(f"Rendement : {frontier_yields[best_idx]:.4f}")
print(f"Volatilité : {frontier_volatility[best_idx]:.4f}")
print(f"Coût : {frontier_cost[best_idx]:.4f}")
print(f"Instabilité rendement (std) : {std_yields[best_idx]:.4f}")
for i, weight in enumerate(w_robust):
    if weight > 1e-4:
        print(f"  {df.columns[i]} : {weight:.4f}")

Portefeuille robuste sélectionné :
Rendement : 0.0770
Volatilité : 0.1563
Coût : 0.0020
Instabilité rendement (std) : 0.0819
  KO : 0.5154
  JNJ : 0.4846
